In [10]:
import geojson
import folium
import overpy
#from shapely.geometry import Point
#import geopandas as gpd
from re import M

camping_icon = '<svg xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" version="1.1" id="Capa_1" x="0px" y="0px" viewBox="0 0 305.601 305.601" style="enable-background:new 0 0 305.601 305.601;" xml:space="preserve"> <g> <g> <g> <path style="fill:#010002;" d="M305.601,211.021l-9.213-1.534l-2.443,14.597C248.954,188.501,148.604,51.347,147.529,49.882     l-3.41-4.621l-2.951,4.895C114.692,94.228,40.292,209.536,11.138,219.306l-2.57-5.901L0,217.147l17.147,39.471l-0.596,0.85h0.977     l1.251,2.872l6.624-2.872h266.139l-0.967-1.212l7.259,1.212l4.445-26.487l0.459-0.85l-0.293-0.205L305.601,211.021z      M192.189,249.007h-85.401c-2.589-5.803-4.143-12.135-4.143-18.905c0-25.852,20.967-46.848,46.828-46.848     c25.891,0,46.848,20.977,46.848,46.848C196.322,236.853,194.778,243.214,192.189,249.007z M288.865,254.137L145.702,73.662     L23.448,247.649l-9.291-21.387C49.222,213.004,128.155,86.109,144.716,58.948c18.71,25.393,106.983,143.739,147.774,173.596     L288.865,254.137z"/> </g> </g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> <g> </g> </svg>'

In [ ]:
api = overpy.Overpass()

query_result = api.query("""
        area[name="England"]->.boundaryarea;
        (
        nwr(area.boundaryarea)[tourism=camp_site];
        );
        out meta;
    """)

In [13]:
import overpass
api = overpass.API()

result = api.get("""
        area[name="England"]->.boundaryarea;
        (
        nwr(area.boundaryarea)[tourism=camp_site];
        );
        out meta;
    """)

SSLError: HTTPSConnectionPool(host='overpass-api.de', port=443): Max retries exceeded with url: /api/interpreter (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)')))

In [8]:
type(query_result)

overpy.Result

In [11]:
with open("./test.geo.json",mode="w") as f:
  cgeojson.dump(query_result,f)

TypeError: Object of type Result is not JSON serializable

In [3]:
m = folium.Map(location=[53,0],
    zoom_start=8,
    tiles='https://{s}.tile-cyclosm.openstreetmap.fr/cyclosm/{z}/{x}/{y}.png',
    #tiles='cartodbpositron',
    attr='Open Street Map Cyclosm'
    )

In [4]:

for site in query_result.nodes:
  html = (
        "<h3><center>" + str(site.tags.get('name')) + "</center></h3>" +
        "<p><Center>" + "<a href='https://www.openstreetmap.org/edit?editor=id&node=" +
        str(site.id) + "#map=16/" +
        str(float(site.lat)) + "/" +
        str(float(site.lon)) +"' target='_blank'>" +
        str(site.id) + "</a> - "+
        "<a href='https://www.google.com/maps/@" + str(site.lat) + "," +
        str(site.lon) + ",16z' target='_blank'> Google Maps </a></center></p>"
        )
  for key in site.tags.keys():
    if key == "website":
      html += ("<b><a href='"+ str(site.tags['website']) + "' target='_blank'>" +
      str(site.tags['website']) + "</a></b><br />")
    else:
      html += key + ": <b>" +str(site.tags[key]) + "</b><br />"

  iframe = folium.IFrame(html=html, width=300, height=300)
  m.add_child(
    folium.Marker(
    location= [float(site.lat), float(site.lon)],
    popup = folium.Popup(html=iframe),
    icon = folium.features.DivIcon(icon_size=(12,12), html=camping_icon)
      )
  )

m

In [14]:
import numpy as np

def gaussian(x, mu, sig):
    return np.exp(-np.power(x - mu, 2.) / (2 * np.power(sig, 2.)))

# Create new chromatogram
chromatogram = MSChromatogram()

# Set raw data (RT and intensity)
rt = range(1500, 500, -100)
i = [gaussian(rtime, 1000, 150) for rtime in rt]
chromatogram.set_peaks([rt, i])

# Sort the peaks according to ascending retention time
chromatogram.sortByPosition()

# Iterate over chromatogram of those peaks
for p in chromatogram:
    print(p.getRT(), p.getIntensity())

# More efficient peak access with get_peaks()
for rt, i in zip(*chromatogram.get_peaks()):
    print(rt, i)

# Access a peak by index
print(chromatogram[2].getRT(), chromatogram[2].getIntensity())

# Add meta information to the chromatogram
chromatogram.setNativeID("Trace XIC@405.2")

# Store a precursor ion for the chromatogram
p = Precursor()
p.setIsolationWindowLowerOffset(1.5)
p.setIsolationWindowUpperOffset(1.5)
p.setMZ(405.2) # isolation at 405.2 +/- 1.5 Th
p.setActivationEnergy(40) # 40 eV
p.setCharge(2) # 2+ ion
p.setMetaValue("description", chromatogram.getNativeID())
p.setMetaValue("peptide_sequence", chromatogram.getNativeID())
chromatogram.setPrecursor(p)

# Also store a product ion for the chromatogram (e.g. for SRM)
p = Product()
p.setMZ(603.4) # transition from 405.2 -> 603.4
chromatogram.setProduct(p)

# Store as mzML
exp = MSExperiment()
exp.addChromatogram(chromatogram)
MzMLFile().store("testfile3.mzML", exp)

# Visualize the resulting data using matplotlib
import matplotlib.pyplot as plt

for chrom in exp.getChromatograms():
    retention_times, intensities = chrom.get_peaks()
    plt.plot(retention_times, intensities, label = chrom.getNativeID())

plt.xlabel('time (s)')
plt.ylabel('intensity (cps)')
plt.legend()
plt.show()

NameError: name 'MSChromatogram' is not defined